## SparkX - GraphFrame
Pour rappel, l'abstraction GraphFrame est sortie en début mars 2016. Le package permet de l'exploiter sous Scala, Python et Java doit être installé en plus de la distribution supérieure ou égale à 1.4.0. Ce Notebook reprend celui de Databriks déroulé sous leur propre environnement. Le scénario concerne un contexte minimaliste de réseau social (7 individus autour de la trentaine, saut un du double et n'ayant aucune relation avec les autres). La représentation en graphe de ce réseau est donnée ci-dessous, à l'exception de Gabby (60 ans) qui n'est pas un adèpte de ce mode de collaboration.

![Social Network Graph](http://p2.pstatp.com/large/33d000f062d8d1b486d)

In [1]:
# le fichier kernel.json pour PySpark sous Jupyter doit-être actulisé prenant en compte le package GraphFrame 
from graphframes import *

In [2]:
# initialisation des sommets du graphe avec le nom et l'âge des individus retenus
vertices = sqlContext.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 36),
  ("g", "Gabby", 60)], ["id", "name", "age"])

In [3]:
# initialisation des arêtes du graphe avec pour attribut le niveau de relation entre individus
edges = sqlContext.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend")
], ["src", "dst", "relationship"])

In [4]:
# création d'un graphframe relatif à l'abstraction du graphe
g = GraphFrame(vertices, edges)
print g

GraphFrame(v:[id: string, name: string, age: bigint], e:[src: string, dst: string, relationship: string])


In [7]:
# recherche de relation entre individus (mono et double sens) à l'aide du simple langage DSL
motifs = g.find("(a)-[e1]->(b); (b)-[e2]->(a)")
motifs.show()

+------------+--------------+--------------+------------+
|          e1|             a|             b|          e2|
+------------+--------------+--------------+------------+
|[c,b,follow]|[c,Charlie,30]|    [b,Bob,36]|[b,c,follow]|
|[b,c,follow]|    [b,Bob,36]|[c,Charlie,30]|[c,b,follow]|
+------------+--------------+--------------+------------+



In [8]:
# filtrage sur les relations trouvées
filtered = motifs.filter("b.age > 30 or a.age > 30")
filtered.show()

+------------+--------------+--------------+------------+
|          e1|             a|             b|          e2|
+------------+--------------+--------------+------------+
|[c,b,follow]|[c,Charlie,30]|    [b,Bob,36]|[b,c,follow]|
|[b,c,follow]|    [b,Bob,36]|[c,Charlie,30]|[c,b,follow]|
+------------+--------------+--------------+------------+



In [9]:
# pipeline de recherche et de filtrages
paths = g.find("(a)-[e]->(b)")\
  .filter("e.relationship = 'follow'")\
  .filter("a.age < b.age")

In [10]:
# The `paths` variable contains the vertex information, which we can extract:
e2 = paths.select("e.src", "e.dst", "e.relationship")

In [11]:
# Construct the subgraph
g2 = GraphFrame(g.vertices, e2)

In [12]:
# affichage des sommets du sous-graphe créé
g2.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+



### Plusieurs algorithmes vont être déroulés avec ce graphe
1. Breadth-first search (BFS)
2. Connected components
3. Strongly connected components
4. Label Propagation Algorithm (LPA)
5. PageRank
6. Shortest paths
7. Triangle count

In [13]:
# 1. Recherche des releations de Ester âgées de moins de 32 ans
paths = g.bfs("name = 'Esther'", "age < 32")

In [14]:
paths.show()

+-------------+------------+------------+
|         from|          e0|          to|
+-------------+------------+------------+
|[e,Esther,32]|[e,d,friend]|[d,David,29]|
+-------------+------------+------------+



In [15]:
# 1.a La recherche peut également être limitée par des filtres de sommets et des longueurs de trajet maximales.
filteredPaths = g.bfs(
  fromExpr = "name = 'Esther'",
  toExpr = "age < 32",
  edgeFilter = "relationship != 'friend'",
  maxPathLength = 3)
filteredPaths.show()

+-------------+------------+------------+------------+--------------+
|         from|          e0|          v1|          e1|            to|
+-------------+------------+------------+------------+--------------+
|[e,Esther,32]|[e,f,follow]|[f,Fanny,36]|[f,c,follow]|[c,Charlie,30]|
+-------------+------------+------------+------------+--------------+



In [16]:
# 2. Recherche d'une composante connexe d'un sous-graphe
result = g.connectedComponents()
result.show()

+---+-------+---+---------+
| id|   name|age|component|
+---+-------+---+---------+
|  a|  Alice| 34|        0|
|  d|  David| 29|        0|
|  b|    Bob| 36|        0|
|  e| Esther| 32|        0|
|  c|Charlie| 30|        0|
|  f|  Fanny| 36|        0|
|  g|  Gabby| 60|        7|
+---+-------+---+---------+



In [17]:
# 3. Calcul de la composante fortement connexe de chaque sommet avec itération
result = g.stronglyConnectedComponents(maxIter=10)
result.show()

+---+-------+---+---------+
| id|   name|age|component|
+---+-------+---+---------+
|  a|  Alice| 34|        0|
|  d|  David| 29|        0|
|  b|    Bob| 36|        2|
|  e| Esther| 32|        0|
|  c|Charlie| 30|        2|
|  f|  Fanny| 36|        5|
|  g|  Gabby| 60|        7|
+---+-------+---+---------+



In [18]:
# 4. Algorithme permettant de détecter des communautés dans des réseaux
result = g.labelPropagation(maxIter=5)
result.show()

+---+-------+---+-----+
| id|   name|age|label|
+---+-------+---+-----+
|  a|  Alice| 34|    2|
|  d|  David| 29|    4|
|  b|    Bob| 36|    4|
|  e| Esther| 32|    2|
|  c|Charlie| 30|    2|
|  f|  Fanny| 36|    4|
|  g|  Gabby| 60|    7|
+---+-------+---+-----+



In [22]:
# 5. PagRank, identification de sommets importants dans un graphe
result = g.pageRank(resetProbability=0.15, tol=0.01)
result.vertices.show()
result.edges.show()

+---+-------+---+-------------------+
| id|   name|age|           pagerank|
+---+-------+---+-------------------+
|  a|  Alice| 34|0.37429242187499995|
|  d|  David| 29|0.27366105468749996|
|  b|    Bob| 36| 2.2131428039184433|
|  e| Esther| 32|  0.309074279296875|
|  c|Charlie| 30|  2.240080617201845|
|  f|  Fanny| 36|0.27366105468749996|
|  g|  Gabby| 60|               0.15|
+---+-------+---+-------------------+

+---+---+------------+------+
|src|dst|relationship|weight|
+---+---+------------+------+
|  c|  b|      follow|   1.0|
|  f|  c|      follow|   1.0|
|  a|  b|      friend|   0.5|
|  a|  e|      friend|   0.5|
|  d|  a|      friend|   1.0|
|  b|  c|      follow|   1.0|
|  e|  d|      friend|   0.5|
|  e|  f|      follow|   0.5|
+---+---+------------+------+



In [23]:
# 6. Calcul de chemins les plus courts
result = g.shortestPaths(landmarks=["a", "d"])
result.show()

+---+-------+---+-------------------+
| id|   name|age|          distances|
+---+-------+---+-------------------+
|  a|  Alice| 34|Map(a -> 0, d -> 2)|
|  d|  David| 29|Map(d -> 0, a -> 1)|
|  b|    Bob| 36|              Map()|
|  e| Esther| 32|Map(d -> 1, a -> 2)|
|  c|Charlie| 30|              Map()|
|  f|  Fanny| 36|              Map()|
|  g|  Gabby| 60|              Map()|
+---+-------+---+-------------------+



In [24]:
# 7. Calcul du nombre de triangles passé par les sommets
result = g.triangleCount()
result.show()

+-----+---+-------+---+
|count| id|   name|age|
+-----+---+-------+---+
|    1|  a|  Alice| 34|
|    0|  b|    Bob| 36|
|    0|  c|Charlie| 30|
|    1|  d|  David| 29|
|    1|  e| Esther| 32|
|    0|  f|  Fanny| 36|
|    0|  g|  Gabby| 60|
+-----+---+-------+---+

